In [9]:
import jieba
import jieba.posseg as jp
from gensim import corpora, models
import pandas as pd
import re

# 引入停用词
from nltk.corpus import stopwords

In [10]:
# 英文句子转词
def get_word_list(s1):
    # 把句子按字分开，中文按字分，英文按单词，数字按空格
    regEx = re.compile('[\\W]*')    # 我们可以使用正则表达式来切分句子，切分的规则是除单词，数字外的任意字符串
    res = re.compile(r"([\u4e00-\u9fa5])")    #  [\u4e00-\u9fa5]中文范围

    p1 = regEx.split(s1.lower())
    str1_list = []
    for str in p1:
        if res.split(str) == None:
            str1_list.append(str)
        else:
            ret = res.split(str)
            for ch in ret:
                str1_list.append(ch)

    list_word1 = [w for w in str1_list if len(w.strip()) > 0]  # 去掉为空的字符

    return  list_word1


In [11]:
# 取数据
stopwords = stopwords.words('english')
def remove_stopwords(ls):  # 去除停用词
    return [word for word in ls if word not in stopwords]

data = pd.read_csv('./重复句子.txt',header=None,names=['text'])
data = data['text'][1:]

documents= data.values

In [12]:
# 去重

data_unique = []
for i in documents:
    if i not in data_unique:
        data_unique.append(i)
print(data_unique)



['ASPIDA rallied its troops to save poor Daisy.', 'Good idea. They have a tough institutional framework against Delaware acquisitions.', 'I like to collect things! I collect incredible', 'President Trump’ s tweet about China’S trade surplus was a joke.', 'Defense Agency Advanced Research Projects. (Defense Planning Research Service)', 'My Sister came up with a Great Idea that Will Make America Strong.', 'A life dedicated to drug trafficking offers you enough in NY.', '[Narrator] May 7th. After five weeks at 6700 meters', 'Tomorrow morning at 6 he will receive two heroin bricks from a Colombian.', 'India’ s attempt to launch a cyber-crime case against an Indian citizen was thwarted.', 'James Gordon JEvsntechnywebsite is not just for you either. It’s for all who want to be informed.', 'A Threat To Supply Chains. A New Challenge To The International Trade System.', 'When Petrov doesn’t get on his phone with him', 'and comes to you through the Crazy Daisy Show.', 'Virtual companies that bu

In [13]:
# 去停用词
words_ls = []
documents = data_unique
for text in documents:
    words = remove_stopwords([w for w in get_word_list(text)])
    print(words)
    words_ls.append(words)

['aspida', 'rallied', 'troops', 'save', 'poor', 'daisy']
['good', 'idea', 'tough', 'institutional', 'framework', 'delaware', 'acquisitions']
['like', 'collect', 'things', 'collect', 'incredible']
['president', 'trump', 'tweet', 'china', 'trade', 'surplus', 'joke']
['defense', 'agency', 'advanced', 'research', 'projects', 'defense', 'planning', 'research', 'service']
['sister', 'came', 'great', 'idea', 'make', 'america', 'strong']
['life', 'dedicated', 'drug', 'trafficking', 'offers', 'enough', 'ny']
['narrator', 'may', '7th', 'five', 'weeks', '6700', 'meters']
['tomorrow', 'morning', '6', 'receive', 'two', 'heroin', 'bricks', 'colombian']
['india', 'attempt', 'launch', 'cyber', 'crime', 'case', 'indian', 'citizen', 'thwarted']
['james', 'gordon', 'jevsntechnywebsite', 'either', 'want', 'informed']
['threat', 'supply', 'chains', 'new', 'challenge', 'international', 'trade', 'system']
['petrov', 'get', 'phone']
['comes', 'crazy', 'daisy', 'show']
['virtual', 'companies', 'buy', 'land', '

C:\Users\15728\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: FutureWarning: split() requires a non-empty pattern match.
  import sys


['device', 'able', 'measure', 'speed', 'stars']
['courts', 'allow']
['found', 'small', 'fracture', 'occipital', 'bone', 'victim']
['back', 'home']
['wanted', 'get', 'look', 'someone', 'alive', 'could', 'say', 'whether', 'infected', 'virus']
['everything', 'amazing', 'best', 'place', 'world']
['go', 'home', 'need', 'come', 'back', 'home']
['virus', 'affect', 'human', 'body', 'affect', 'immune', 'system', 'prevent', 'spreading']
['athlete', 'someone', 'whose', 'life', 'dedicated', 'pursuit', 'excellence']
['new', 'study', 'shows', 'drugs', 'sold', 'street', 'far', 'potent', 'originally', 'intended', 'manufacturers']
['said', 'would', 'able', 'watch', 'tv', 'internet', 'well']
['go', 'foreign', 'country']
['girl', 'would', 'give', 'anything', 'go', 'beach']
['husband', 'wish', 'talk', 'reporters', 'illness']
['seen']
['reading', 'email', 'good', 'way', 'get', 'things', 'done']
['number', 'people', 'want', 'watch', 'movie', 'increasing']
['one', 'clock']
['must', 'everything', 'fight', 'vi

In [14]:
# 生成语料词典
dictionary = corpora.Dictionary(words_ls)
# 生成稀疏向量集
corpus = [dictionary.doc2bow(words) for words in words_ls]


In [15]:
# LDA模型，num_topics设置聚类数，即最终主题的数量
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
# 展示每个主题的前5的词语
for topic in lda.print_topics(num_words=5):
    print(topic)

(0, '0.017*"go" + 0.010*"know" + 0.010*"see" + 0.010*"one" + 0.009*"way"')
(1, '0.017*"take" + 0.015*"virus" + 0.007*"leave" + 0.007*"going" + 0.007*"year"')
(2, '0.021*"want" + 0.013*"learn" + 0.013*"like" + 0.010*"know" + 0.009*"take"')
(3, '0.009*"day" + 0.008*"year" + 0.008*"one" + 0.006*"use" + 0.006*"buy"')
(4, '0.020*"one" + 0.006*"million" + 0.006*"know" + 0.006*"2" + 0.005*"covid"')
(5, '0.014*"get" + 0.010*"give" + 0.009*"back" + 0.008*"virus" + 0.008*"home"')
(6, '0.014*"want" + 0.014*"time" + 0.014*"us" + 0.011*"one" + 0.010*"see"')
(7, '0.009*"dead" + 0.008*"place" + 0.008*"people" + 0.008*"system" + 0.008*"going"')
(8, '0.013*"good" + 0.008*"going" + 0.008*"would" + 0.007*"back" + 0.007*"time"')
(9, '0.015*"time" + 0.009*"china" + 0.008*"much" + 0.008*"found" + 0.006*"family"')


In [16]:
# 推断每个语料库中的主题类别， 按主题存入文件
print('推断：')
res = {}
for e, values in enumerate(lda.inference(corpus)[0]):
    topic_val = 0
    topic_id = 0
    for tid, val in enumerate(values):
        if val > topic_val:
            topic_val = val
            topic_id = tid
            
    with open("{}".format(topic_id),"a") as file:  
      file.write(str(documents[e]) + "\n")

推断：
